IMPORT AND SETTINGS

In [243]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
import matplotlib.pyplot as plt
import pandas as pd
import cv2 
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
import random

try:
    tf.config.experimental.enable_op_determinism()
    print("✅ Op Determinism Abilitato!")
except AttributeError:
    print("⚠️ Attenzione: La tua versione di TF è troppo vecchia per enable_op_determinism.")

def reset_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
SEEDS = [555]

2.21.0-dev20251210
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ Op Determinism Abilitato!


DATASET LOADING AND MODELLING

In [244]:
def load_dataset():
    folder='dataset/images'
    data=[]
    for filename in sorted(os.listdir(folder)):
        img_path=os.path.join(folder,filename)
        img=cv2.imread(img_path) #opencv save in bgr
        data.append({
            'image':img,
            'filename':filename
        })

    print(len(data),'images loaded')
    print('file name is: ',data[0]['filename'], 'shape of the image is:  ', data[0]['image'].shape )

    label=pd.read_csv('dataset/raw/bbx_annotations.csv')
    print(label.shape, label.iloc[0]['filename'])
    #images order is random, and for 1 image you can have more class

    print('we have', len(label['class'].unique()), 'different classes')

    #replace biggger img with half sized ones
    #cv2.imwrite('resize_image/last_img_pre_downsampling.jpg',data[-100]['image'])
    for i,item in enumerate(data):
        if "upper" in item["filename"].lower():
            data[i]['image']=cv2.resize(
                data[i]['image'],
                (data[i]['image'].shape[1]//2,data[i]['image'].shape[0]//2)
                ,interpolation=cv2.INTER_AREA
            )
    #cv2.imwrite('resize_image/last_img_post_downsampling.jpg',data[-100]['image'])
       
    return data, label

FROM THE PURE DATASET TO THE TRAIN AND TEST DATA AND LABEL

In [245]:
def dataset_modelling(dataset,annotation):   
    dataset_df = pd.DataFrame(dataset) 
    label_map={'goalpost':0,
               'ball':1,
               'robot':2,
               'goalspot':3,
               'centerspot':4}
    def get_vector(classes_found):
        vec=np.zeros(5,dtype=int)

        for c in classes_found:
            if c in label_map:
                vec[label_map[c]]=1
        return list(vec)
    
    grouped = annotation.groupby('filename')['class'].apply(list).reset_index()
    grouped['label']=grouped['class'].apply(get_vector)
    final_annotation=grouped[['filename','label']]

    final_dataset= pd.merge(dataset_df, final_annotation[['filename', 'label']], on='filename', how='inner')
    final_dataset.to_csv('csv/temp/final_dataset.csv')
    final_dataset=final_dataset.drop(columns=['filename'])
    df_train, df_test = train_test_split(final_dataset, test_size=0.2, random_state=42)
    x_train = np.array(df_train['image'].tolist()).astype('float32') 
    y_train = np.array(df_train['label'].tolist()).astype('float32')
    
    x_test = np.array(df_test['image'].tolist()).astype('float32') 
    y_test = np.array(df_test['label'].tolist()).astype('float32')
    return x_train, y_train,x_test,y_test
    

COMPUTING THE WEIGHTS FOR EACH CLASS, TO GIVE MORE IMPORTANCE TO UNDER-REPRESENTED CLASS

In [246]:
def compute_class_weight(y_train):
    counts = np.sum(y_train, axis=0)
    tot_samples=np.sum(counts)
    num_class=5
    class_weight_dict={}

    for i,count in enumerate(counts):
        weight=tot_samples/(num_class*count)
        class_weight_dict[i]=weight

        
    return class_weight_dict


CREATE ALL DIFFERENT COMBINATION OF HYPERPARAMETER, WITHOUT USING GRIDSEARCH

In [247]:
def create_hyperparam_combination():
    param_grid = {
    'batch_size': [64,16],
    'layer_number':[4,5],
    'kernel_dim': [3,7],
    'pool_dim': [2,3], 
    'lr': [0.0001,0.001],
    'fc1' : [128],
    'fc2': [128]     
}

    #every possible combination
    keys, values = zip(*param_grid.items())
    combinations = list(itertools.product(*values))
    combinations_dicts = [dict(zip(keys, v)) for v in combinations]
    return combinations_dicts

MODEL BUILIDNG, 
kernel dimesnsion, pooling dimension, fc layers dimension, number of conv layer and learning rate have different combination.
instead, i fixed:
pooling stride=2 
pooling type: avg pooling
number of kernel per layer: 16, 32, 64...
last pooling: glob avg pool

In [248]:
def build_model(layer_num,kernel_dim,pool_dim,fc1,fc2):
    model=models.Sequential()
    model.add(tf.keras.Input(shape=(240,320,3)))

    for i in range(layer_num):
        kernel_number=16*(2**i)
        model.add(layers.Conv2D(kernel_number,(kernel_dim,kernel_dim),activation='relu',padding='same'))
        model.add(layers.AveragePooling2D((pool_dim,pool_dim),strides=2,padding='same'))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(fc1,activation='relu'))
    model.add(layers.Dense(fc2,activation='relu'))
    model.add(layers.Dense(5,activation='sigmoid'))

    #model.summary()
    return model


SAVING THE CSV FILE CONTAINING F1S OF EACH SEARCHED MODEL

In [249]:
def saving_csv(report_dict,param,target_names,all_results,seed):
    current_result = param.copy()    
    for class_name in target_names:
        current_result[f'f1_{class_name}'] = report_dict[class_name]['f1-score']
    current_result['f1_macro_avg'] = report_dict['macro avg']['f1-score']

    all_results.append(current_result)
    pd.DataFrame(all_results).to_csv(f'csv/f1_search/search5Nonorm_{seed}.csv', index=False)



MAIN

In [250]:
dataset, annotation=load_dataset()
x_train,y_train, x_test, y_test=dataset_modelling(dataset, annotation)
combination=create_hyperparam_combination()

all_results = []
bestf1=0
for seed in SEEDS:
    print(f"\n ================== INIZIO CICLO CON SEED: {seed} ================== ")
    for i,param in enumerate(combination):

        
        print(f"\nTRAINING RUN {i+1}/{len(combination)}")
        tf.keras.backend.clear_session()
        reset_seeds(seed)
        model=build_model( param['layer_number'],param['kernel_dim'],param['pool_dim'],param['fc1'],param['fc2'])
        opt=tf.keras.optimizers.Adam(learning_rate=param['lr'])
        model.compile(optimizer=opt,
                    loss='binary_crossentropy',
                    metrics=[tf.keras.metrics.Precision(name='precision'),
                            tf.keras.metrics.Recall(name='recall'),
                            ])
        early_stop=EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
        )
        reset_seeds(seed)
        #class_weights_dict=compute_class_weight(y_train)
        history=model.fit(
            x_train,y_train,
            epochs=120,
            batch_size=param['batch_size'], 
            validation_split=0.2, 
            callbacks=[early_stop]
            )

        y_pred=model.predict(x_test)
        predictions_binary = (y_pred > 0.5).astype(int)
        target_names = ['goalpost','ball','robot','goalspot','centerspot']
        report_dict = classification_report(y_test, predictions_binary, target_names=target_names, output_dict=True)
        f1_macro = report_dict['macro avg']['f1-score']
        if f1_macro>bestf1:
            best_report_dict=report_dict
            best_param=param
            bestf1=f1_macro
        saving_csv(report_dict,param,target_names,all_results,seed)


    df_report = pd.DataFrame(best_report_dict).transpose()
    df_report = df_report.round(2)
    df_report['support'] = df_report['support'].astype(int)
    csv_path = f'csv/report/report_bestcomb5Nonorm_{seed}.csv'
    df_report.to_csv(csv_path)
    print(best_param)




2452 images loaded
file name is:  lower_100056_jpg.rf.ec9852c66b4eee4a185317210a378f16.jpg shape of the image is:   (240, 320, 3)
(8125, 8) upper_604302_jpg.rf.6215ee30a829ec658154eb4d067dfdf5.jpg
we have 5 different classes

 ================== INIZIO CICLO CON SEED: 555 ================== 

TRAINING RUN 1/32
Epoch 1/120
21/22 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.7148 - precision: 0.6044 - recall: 0.5482

E0000 00:00:1765536105.818973   44732 node_def_util.cc:682] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.5776 - precision: 0.6961 - recall: 0.4948 - val_loss: 0.5055 - val_precision: 0.7762 - val_recall: 0.4732
Epoch 2/120
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.4643 - precision: 0.8374 - recall: 0.4994 - val_loss: 0.4727 - val_precision: 0.8184 - val_recall: 0.4905
Epoch 3/120
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.4498 - precision: 0.8467 - recall: 0.5056 - val_loss: 0.4656 - val_precision: 0.8033 - val_recall: 0.5078
Epoch 4/120
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.4440 - precision: 0.8374 - recall: 0.5185 - val_loss: 0.4622 - val_precision: 0.8000 - val_recall: 0.5043
Epoch 5/120
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.4395 - precision: 0.8310 - recall: 0.5281 - val_loss: 0.4569 - val_precision: 0.7829 - val_recall: 0.5233
Epoch 6/120
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.4351 - precision: 0.8281 - recall: 0.5314 - val_loss: 0.4551 - val_precision: 0.7626 - val_recall: 0.5492
Epoc

E0000 00:00:1765536170.202789   44732 node_def_util.cc:682] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}



TRAINING RUN 2/32


...

/home/simone/Desktop/ml_homework2/ml_hw2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Epoch 1/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 0.5948 - precision: 0.7746 - recall: 0.4677 - val_loss: 0.5093 - val_precision: 0.7756 - val_recall: 0.4715
Epoch 2/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4858 - precision: 0.8098 - recall: 0.5031 - val_loss: 0.4841 - val_precision: 0.7700 - val_recall: 0.5320
Epoch 3/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - loss: 0.4836 - precision: 0.8109 - recall: 0.5156 - val_loss: 0.4765 - val_precision: 0.8118 - val_recall: 0.4767
Epoch 4/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.4622 - precision: 0.8438 - recall: 0.5081 - val_loss: 0.4751 - val_precision: 0.8234 - val_recall: 0.4991
Epoch 5/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.4564 - precision: 0.8432 - recall: 0.5148 - val_loss: 0.4600 - val_precision: 0.8471 - val_recall: 0.4784
Epoch 6/120
88/88 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.4469 - precision: 0.8397 - recall: 0.5277 - val_loss: 0.4513 - val_precision: 0.8498 - val_recall:

E0000 00:00:1765538517.176674   44732 node_def_util.cc:682] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
{'batch_size': 64, 'layer_number': 5, 'kernel_dim': 3, 'pool_dim': 3, 'lr': 0.001, 'fc1': 128, 'fc2': 128}


/home/simone/Desktop/ml_homework2/ml_hw2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
